In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [43]:
wiki = pd.read_csv('../data/wikipedia.csv')
uncy = pd.read_csv('../data/uncyclopedia.csv')

In [44]:
wiki.head(10)

,Field1
0,京都大学（きょうとだいがく、英語: Kyoto University）は、京都府京都市左京区...
1,京都大学は、日本で2番目に創設された帝国大学たる京都帝国大学の流れを汲んでいる国立大学で、入...
2,創立以来「自由の学風」を建学の精神としている。国立大学法人となった後に制定された京都大学の基...
3,京都市内の繁華街から離れた古都の風情を残す落ち着いた環境の中にあり、何事も学生の自主性に任せ...
4,京都大学や（前身の）京都帝国大学では、それらの自由の拡大あるいは縮小を巡る攻防戦が繰り返され...
5,京都大学は学部・研究科以外に19の附置研究所・研究センターを設置している[5]。2010年3...
6,タイムズ・ハイアー・エデュケーションの『THE世界大学ランキング 2021』（2020年）で...
7,2019年10月10日現在、卒業生から8人、ゆかりのある人物も含めると11人のノーベル賞受賞...
8,（沿革節の主要な出典は公式サイト[11][12][13][14][15][16][17]）\n
9,京都大学の起源は文久元年8月16日（1861年9月20日）に長崎に設立された長崎養生所（その...


In [45]:
uncy.head(10)

,Field1
0,京都大学（きょうとだいかく、英称：Kyoto University）は、京都府京都市を本拠地...
1,"「1人の天才と99,999人の狂人を生み出す大学」と呼ばれ、99,999人の中には態度がデカ..."
2,今日、カステラ医学の権威として知られる長崎大学医学部を本家と仰ぐ、ハラタイラ・烏賊学部門が新...
3,建学当初から自由を基調とした考えが根底にある。部落開放同盟と利権を欲しいままにし、京都市役所...
4,また、官僚養成のトンキン大学や実学を推奨する神戸大学に対して、ニート養成や空想科学、エロゲ制...
5,永遠の校長こと折田彦市尊師を称えた銅像があったが、うら若き京大生達から溢れ出る芸術の息吹が尊...
6,出町柳のよそもん下宿生はテレビ大阪が写らない。\n
7,牛車は大学側が用意するので、自前で購入する必要は無い。ただ、烏帽子や狩衣については入学前に購...
8,学内には、独立国家・吉田寮が存在する。クスノキ前で麻雀をやったり、炬燵で鍋をつついている連中...
9,お隣のソースの効いたたこ焼き大学の総長は京大卒の坂田利夫である。\n


In [47]:
# Remove new line character
def parse(x):
    x = re.sub("\n","", x)
    x = re.sub("「|」","",x)
    x = re.sub("『|』","",x)
    x = re.sub("（.*?）","",x)
    x = re.sub("\[.*?\]","",x)
    x = re.sub("[0-9]+","0",x)
    x = re.sub("・","、",x)
    return x

In [48]:
wiki['Field1'] = wiki['Field1'].map(lambda x: parse(x))
uncy['Field1'] = wiki['Field1'].map(lambda x: parse(x))

In [49]:
wiki.head(10)

,Field1
0,京都大学は、京都府京都市左京区吉田本町に本部を置く日本の国立大学である。0年に設置された。大...
1,京都大学は、日本で0番目に創設された帝国大学たる京都帝国大学の流れを汲んでいる国立大学で、入...
2,創立以来自由の学風を建学の精神としている。国立大学法人となった後に制定された京都大学の基本理...
3,京都市内の繁華街から離れた古都の風情を残す落ち着いた環境の中にあり、何事も学生の自主性に任せ...
4,京都大学や京都帝国大学では、それらの自由の拡大あるいは縮小を巡る攻防戦が繰り返されてきた。滝...
5,京都大学は学部、研究科以外に0の附置研究所、研究センターを設置している。0年0月現在、国内の...
6,タイムズ、ハイアー、エデュケーションのTHE世界大学ランキング 0では、世界第0位、アジア第...
7,0年0月0日現在、卒業生から0人、ゆかりのある人物も含めると0人のノーベル賞受賞者を輩出して...
8,
9,京都大学の起源は文久元年0月0日に長崎に設立された長崎養生所まで遡る。長崎精得館の理化学部門...


In [26]:
wiki['lenght'] = wiki['Field1'].map(lambda x: len(x))
uncy['lenght'] = uncy['Field1'].map(lambda x: len(x))

In [27]:
wiki = wiki[wiki['lenght'] > 100]
uncy = uncy[uncy['lenght'] > 100]

In [28]:
wiki['label'] = 1
uncy['label'] = 0

In [29]:
data = pd.concat([wiki, uncy])

In [30]:
y, X = data['label'], data['Field1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [31]:
train = pd.DataFrame({'label': y_train, 'text': X_train})
test = pd.DataFrame({'label': y_test, 'text': X_test})

In [32]:
len(train)

742

In [33]:
len(test)

248

In [34]:
test.to_csv('../data/test.txt', sep='\t', index=False, header=False)
train.to_csv('../data/train.txt', sep='\t', index=False, header=False)

In [35]:
test.label.mean()

0.6330645161290323

In [36]:
train.label.mean()

0.6347708894878706

In [37]:
test

,label,text
413,1,1886年に講義録を発行し通信教育を開始したことがその起源である。当時の印刷部門は独立し、そ...
646,1,21世紀プログラムは、専門性の高いゼネラリストの育成を掲げて、2000年に学士課程に設置され...
963,1,設立に際しては、旧鳥取藩主・池田輝知や旧島原藩主・松平忠和らから支援を受け、千代田区有楽町数...
594,1,東京高等工芸学校のテクノロジー部門は現在の東京工業大学。同校の存した場所である港区芝浦の東京...
156,1,学生による自治を原則としているが、大学当局ともその支援を受けながら良好な関係を保っている。創...
...,...,...
99,1,ここまでの経緯としては、盧溝橋事件をきっかけに日中戦争が勃発すると、大学内でも配属将校はもち...
992,1,国内のOB・OG連合組織である連合駿台会は、1953年に経済界OB・OGの集いとして設立され...
597,1,1890年、第一高等中学校医学部に設置された薬学科を起源とする歴史がある。1949年に新制総...
438,0,早稲田大学には、学校法人早稲田大学が設置する以下の附属学校が存在している。学内での地位は学部...


In [39]:
test[test['label'] == 0]

,label,text
434,0,早稲田大学戸山キャンパスや戸山公園、早稲田大学西早稲田キャンパス、学習院女子大学など早稲田の...
224,0,大学当局の公認サークル。団体設立には必ず顧問教官を置く必要があるほか、本郷キャンパスにサーク...
445,0,2010年には、大学院・先進理工学研究科に、日本初となる他大学との共同専攻、共同先進健康科学...
582,0,複数学問分野を包括する行動科学科を除いて、一学科の定員は約40名と高校の一クラス程度である。...
265,0,1900年1月29日、第14回帝国議会において九州東北帝国大学設置建議案が採択され、議会とし...
...,...,...
544,0,一橋大学の同窓組織は一般社団法人如水会といい、文部科学省所管の公益法人となっていた。もともと...
78,0,慶應義塾大学は、中津藩士の福澤諭吉が藩命により江戸築地鉄砲洲の中津藩中屋敷内に、1858年に...
553,0,学術総合センタービルを指す。2012年5月に一橋記念講堂の運営が独立行政法人国立大学財務・経...
508,0,体育系の実績のある高校生に対して推薦入学など入学優遇を行う制度は存在しない。ただし、個人競技...
